### Import Modules

In [ ]:
import glob
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
import os
from nibabel.streamlines import save as save_trk
from nibabel.streamlines import Tractogram
from os import path
from dipy.tracking.local import ParticleFilteringTracking
from dipy.core.gradients import gradient_table
from dipy.data import get_sphere, Sphere, HemiSphere
from dipy.direction import (peaks_from_model, ProbabilisticDirectionGetter)
from dipy.io import read_bvals_bvecs
from dipy.io.image import save_nifti
from dipy.io.streamline import save_trk, load_trk
from dipy.reconst.peaks import reshape_peaks_for_visualization
from dipy.reconst.dti import TensorModel, fractional_anisotropy, mean_diffusivity
from dipy.reconst.csdeconv import (ConstrainedSphericalDeconvModel,auto_response, recursive_response)
from dipy.reconst.shm import sh_to_sf
from dipy.segment.mask import median_otsu
from dipy.tracking.local import (LocalTracking, ThresholdTissueClassifier, ActTissueClassifier, CmcTissueClassifier)
from dipy.tracking.streamline import Streamlines
from dipy.tracking.utils import random_seeds_from_mask, move_streamlines
from dipy.viz import actor, window
from dipy.tracking.utils import target, density_map

In [ ]:
os.chdir('/space/hemera/1/users/cmaffei/hcp_processing')
wd = ('/space/hemera/1/users/cmaffei/hcp_processing')

In [ ]:
sphere = get_sphere('symmetric724')

## CSD for b=1k

In [ ]:
for sub in sorted(glob.glob(os.path.join(wd, 'mgh_*'))):
    print('Loading data for'+str(sub)) #loading data
    img = nib.load(os.path.join(sub,'1k/data_corr.nii.gz'))
    data = img.get_data()
    bvals, bvecs = read_bvals_bvecs(os.path.join(sub, '1k/bvals'), os.path.join(sub, '1k/bvecs'))
    gtab = gradient_table(bvals, bvecs)
    mask = nib.load(os.path.join(sub, 'mask_er.nii.gz')).get_data()
    #recursive estimation
    print('Starting recursive estimation for'+str(sub))
    tenmodel = TensorModel(gtab)
    tenfit = tenmodel.fit(data, mask)
    FA = fractional_anisotropy(tenfit.evals)
    MD = mean_diffusivity(tenfit.evals)
    wm_mask = (np.logical_or(FA >= 0.4, (np.logical_and(FA >= 0.3, MD >= 0.0011))))
    response = recursive_response(gtab, data, mask=wm_mask, sh_order=8,
                                  peak_thr=0.01, init_fa=0.08,
                              init_trace=0.0021, iter=8, convergence=0.001, parallel=True, nbr_processes=4)
    #check the shape of the RF,and take pic:
    response_signal = response.on_sphere(sphere)
    response_signal = response_signal[None, None, None, :]
    response_actor = actor.odf_slicer(
    response_signal, sphere=sphere, colormap='plasma')
    interactive = False
    ren = window.Renderer()
    ren.add(response_actor)
    print('Saving illustration for'+str(sub))
    window.record(ren, out_path='1k/csd_recursive_response.png', size=(200, 200))
    if interactive:
        window.show(ren)
    print('Computing FODFs for'+str(sub))
    csd_model = ConstrainedSphericalDeconvModel(gtab, response, sh_order=8)
    csd_peaks = peaks_from_model(model=csd_model,
                             data=data,
                             sphere=sphere,
                             relative_peak_threshold=.5,
                             sh_order=8,
                             min_separation_angle=25,
                             mask=mask, return_odf=True, return_sh=True)
    print('Saving FODFs for'+str(sub))
    nib.save(nib.Nifti1Image(csd_peaks.shm_coeff.astype(np.float32),
                                  img.affine, img.header), os.path.join(sub, '1k/odfs_recursive_csd.nii.gz'))

In [ ]:
#checking ODFs
interactive = True
sphere = get_sphere('symmetric724')

for sub in glob.glob(os.path.join(wd, 'mgh_*')):
    print('Loading ODFs for'+str(sub))
    #load files 
    img = nib.load(os.path.join(sub, 'odfs_recursive_csd.nii.gz'))
    data = img.get_data()
    data=sh_to_sf(data, sphere, sh_order=6)
    ren = window.Renderer()
    fodf_spheres = actor.odf_slicer(data, sphere=sphere, scale=0.9, norm=False, colormap='plasma')
    ren.add(fodf_spheres)
    print('Saving illustration as csd_odfs.png')
    window.record(ren, out_path=os.path.join(sub,'1k/csd_fodfs.png'), size=(600, 600))
    if interactive:
        window.show(ren)

## CSD for b=10k

### Obtain Response function

In [ ]:
%%script tcsh
mrtrixenv
foreach sub (mgh_????)
    set input =  $sub/wmparc2diff.nii.gz
    set output = $sub/body_cc.nii.gz
    mri_extract_label $input 253 254 $output
end

In [4]:
for sub in sorted(glob.glob(os.path.join(wd, 'mgh_*'))):
    print('Loading data for'+str(sub)) #loading data
    img = nib.load(os.path.join(sub, '10k/data_corr.nii.gz'))
    data = img.get_data()
    bvals, bvecs = read_bvals_bvecs(os.path.join(sub, '10k/bvals'), os.path.join(sub, '10k/bvecs'))
    gtab = gradient_table(bvals, bvecs)
    mask = nib.load(os.path.join(sub, 'nodif_brain_mask.nii.gz')).get_data()
    #recursive estimation
    print('Starting recursive estimation for'+str(sub))
    wm_mask=nib.load(os.path.join(sub,"body_cc.nii.gz")).get_data()
    wm_mask=wm_mask>0.7
    response = recursive_response(gtab, data, mask=wm_mask, sh_order=8,
                                  peak_thr=0.1, init_fa=0.1,
                              init_trace=0.0021, iter=20, convergence=0.01, parallel=True, nbr_processes=4)

   
    print('Computing FODFs for'+str(sub))
    csd_model = ConstrainedSphericalDeconvModel(gtab, response, sh_order=8)
    csd_peaks = peaks_from_model(model=csd_model,
                             data=data,
                             sphere=sphere,
                             relative_peak_threshold=.5,
                             sh_order=8,
                             min_separation_angle=25,
                             mask=mask, return_sh=True)
    print('Saving FODFs for'+str(sub))
    nib.save(nib.Nifti1Image(csd_peaks.shm_coeff.astype(np.float32),
                                  img.affine, img.header), os.path.join(sub, '10k/odfs_01_01_body_cc.nii.gz'))

Loading data for/space/hemera/1/users/cmaffei/hcp_processing/mgh_1001
Starting recursive estimation for/space/hemera/1/users/cmaffei/hcp_processing/mgh_1001
Computing FODFs for/space/hemera/1/users/cmaffei/hcp_processing/mgh_1001
Saving FODFs for/space/hemera/1/users/cmaffei/hcp_processing/mgh_1001
Loading data for/space/hemera/1/users/cmaffei/hcp_processing/mgh_1002
Starting recursive estimation for/space/hemera/1/users/cmaffei/hcp_processing/mgh_1002
Computing FODFs for/space/hemera/1/users/cmaffei/hcp_processing/mgh_1002
Saving FODFs for/space/hemera/1/users/cmaffei/hcp_processing/mgh_1002
Loading data for/space/hemera/1/users/cmaffei/hcp_processing/mgh_1003
Starting recursive estimation for/space/hemera/1/users/cmaffei/hcp_processing/mgh_1003
Computing FODFs for/space/hemera/1/users/cmaffei/hcp_processing/mgh_1003
Saving FODFs for/space/hemera/1/users/cmaffei/hcp_processing/mgh_1003
Loading data for/space/hemera/1/users/cmaffei/hcp_processing/mgh_1004
Starting recursive estimation 